In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
 

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [17]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break





Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [20]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
for t in range(1):
    test(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 1.170089  [    0/60000]
loss: 1.186221  [ 6400/60000]
loss: 1.024515  [12800/60000]
loss: 1.162460  [19200/60000]
loss: 1.054054  [25600/60000]
loss: 1.082970  [32000/60000]
loss: 1.144745  [38400/60000]
loss: 1.090044  [44800/60000]
loss: 1.140826  [51200/60000]
loss: 1.087700  [57600/60000]
Epoch 2
-------------------------------
loss: 1.170089  [    0/60000]
loss: 1.186221  [ 6400/60000]
loss: 1.024515  [12800/60000]
loss: 1.162460  [19200/60000]
loss: 1.054054  [25600/60000]
loss: 1.082970  [32000/60000]
loss: 1.144745  [38400/60000]
loss: 1.090044  [44800/60000]
loss: 1.140826  [51200/60000]
loss: 1.087700  [57600/60000]
Epoch 3
-------------------------------
loss: 1.170089  [    0/60000]
loss: 1.186221  [ 6400/60000]
loss: 1.024515  [12800/60000]
loss: 1.162460  [19200/60000]
loss: 1.054054  [25600/60000]
loss: 1.082970  [32000/60000]
loss: 1.144745  [38400/60000]
loss: 1.090044  [44800/60000]
loss: 1.140826  [51200/60000]
loss: 1.08

In [12]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")


Saved PyTorch Model State to model.pth


In [21]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>

In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


Predicted: "Ankle boot", Actual: "Ankle boot"


In [15]:
tensor = torch.rand(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

First row: tensor([0.0497, 0.8410, 0.6423, 0.9328])
First column: tensor([0.0497, 0.0610, 0.8109, 0.6977])
Last column: tensor([0.9328, 0.0718, 0.1615, 0.3736])
tensor([[0.0497, 0.0000, 0.6423, 0.9328],
        [0.0610, 0.0000, 0.6985, 0.0718],
        [0.8109, 0.0000, 0.8353, 0.1615],
        [0.6977, 0.0000, 0.0934, 0.3736]])


In [3]:
import pandas